In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

Once again we use openQmin to simulate a homeotropic cell. The molecules lie along the z-axis, or the assumed optical axis, as expected.

IN: openQmin.cpp

#include "addObjectsToOpenQmin.h"

    scalar targetNematicValue = S0;
    scalar anchoringStrength1 = 12.919; /*This is The NON_DIM value for homeotropic anchoring*/
    boundaryObject homeotropicBoundary1(boundaryType::homeotropic,anchoringStrength1,targetNematicValue);
    sim->createWall(2,0,homeotropicBoundary1);

But now we want to add unique anchoring ferromagnetic objects to match the geometries of interest used to create bespoke director patterns. Once again, we begin with a handy guide developed by Dan Beller found at https://github.com/sussmanLab/open-Qmin/blob/master/tools/demo_boundaryHelper.ipynb

In [2]:
#Big Thanks to Dan Beller for this file
import boundaryHelper as bh

For our purposeses, we used the shapes available at the above repository, and also added a few key shapes. These shapes are cubes, flat cylinders, and washers. Each shape requires a member_func and a normal_func which specify the points included in the boundary, and the normals to the boundary respectively. 

Setting the scene for our object:

In [3]:
Lx = 240
Ly = 240
Lz = 28
X, Y, Z = np.meshgrid(np.arange(Lx), np.arange(Ly), np.arange(Lz))

and defining a center for our boundary objects as well as relevant parameters:

In [4]:
#C will be centered where X,Y,Z, go from 0 --> L 
Cx, Cy, Cz = np.array([((Lx-1)/2), ((Ly-1)/2), ((Lz-1)/2)])
l = 39 #cube length
w = 39 #cube width
h = 19 #cube height

If you are anything like me, you would really benefit from a sanity check here:

In [5]:
print('In this notebook:')

print('The lattice sites go from')

print('X: 0 to ' +str(Lx-1))
print('Y: 0 to ' +str(Ly-1))
print('Z: 0 to ' +str(Lz-1))

print('Cube is centered at')
print(Cx,Cy,Cz)

print('With L, W, H')
print(int(l),int(w),int(h))

print('The walls of the Cube extend over')

print('X: ' + str(Cx - (l/2)) + ' to ' + str(Cx + (l/2)))
print('Y: ' + str(Cy - (w/2)) + ' to ' + str(Cy + (w/2)))
print('Z: ' + str(Cz - (h/2)) + ' to ' + str(Cz + (h/2)))

In this notebook:
The lattice sites go from
X: 0 to 239
Y: 0 to 239
Z: 0 to 27
Cube is centered at
119.5 119.5 13.5
With L, W, H
39 39 19
The walls of the Cube extend over
X: 100.0 to 139.0
Y: 100.0 to 139.0
Z: 4.0 to 23.0


First lets generate a boundary file for a cube using cube functions

In [6]:
#Cube centered at (Cx,Cy,Cz) with (l w h) 
def cube_member_func(X,Y,Z):
    bool_1 = (np.absolute(X-Cx)<=(l/2))
    bool_2 = (np.absolute(Y-Cy)<=(w/2))
    bool_3 = (np.absolute(Z-Cz)<=(h/2))
    return(bool_1*bool_2*bool_3)

def cube_normal_func(X, Y, Z):
    
    z_count = 0
    y_count = 0
    x_count = 0
    
    returnX = np.zeros(len(X))
    returnY = np.zeros(len(Y))
    returnZ = np.zeros(len(Z))
    
    #i realize there might be a more efficiant way to do this.... maybe with functional forms.
    for i in range(len(X)):
        
        if int(Z[i]) == int(Cz + (h/2)):
            returnX[i] = 0
            returnY[i] = 0
            returnZ[i] = 1 
            z_count += 1
            #print('+z')
                    
        elif int(Z[i]) == int(Cz - (h/2)):
            returnX[i] = 0
            returnY[i] = 0
            returnZ[i] = -1 
            z_count += 1
            #print('-z')
                    
        elif int(Y[i]) == int(Cy + (w/2)):
            returnX[i] = 0
            returnY[i] = 1
            returnZ[i] = 0 
            y_count += 1
            #print('y')
            
        elif int(Y[i]) == int(Cy - (w/2)):
            returnX[i] = 0
            returnY[i] = -1
            returnZ[i] = 0 
            y_count += 1
            #print('-y')
            
        elif int(X[i]) == int(Cx + (l/2)):
            returnX[i] = 1
            returnY[i] = 0
            returnZ[i] = 0 
            x_count += 1
            #print('x')
            
        elif int(X[i]) == int(Cx - (l/2)):
            returnX[i] = -1
            returnY[i] = 0
            returnZ[i] = 0
            x_count += 1
            #print('-x')
                        
        else:
            #print(X[i],Y[i],Z[i])
            returnX[i] = 0
            returnY[i] = 0
            returnZ[i] = 0
            #print('else')
            
    return(returnX,returnY,returnZ)

The next part is easy, because it was made easy by Dan Beller. 

In [7]:
sc = bh.Scene(Lx, Ly, Lz)

In [8]:
ac = bh.OrientedAnchoringCondition(strength = 12.9, S0 = 0.53) #set anchoring strength here
bo = bh.BoundaryObject(ac)

In [9]:
bo.member_func = cube_member_func
bo.normal_func = cube_normal_func
sc.boundary_objects.append(bo)
sc.to_file('boundaries/cuboid_12_28_240_240_28_39_39_19.txt')

To generate washer or cylinder geometries, you can navigate to /boundaries/Boundary_Shape